# Food and Agriculture Organization
## L'étude de santé publique

La MISSION: réaliser une étude de grande ampleur sur le thème de la sous-nutrition dans le monde.

_____________________

Je commence avec l'importation des libraries et des données que je vais utiliser pour l'étude de santé public.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import os

os.chdir("C:/Users/majai/Dropbox/DATA ANALYST formation/Project 4/Livrables/DAN-P4-FAO")
aide_alimentaire = pd.read_csv('CSV/aide_alimentaire.csv')
dispo_alimentaire = pd.read_csv('CSV/dispo_alimentaire.csv')
population = pd.read_csv('CSV/population.csv')
sous_nutrition = pd.read_csv('CSV/sous_nutrition.csv')

## L'exploration des donnees

### Population

In [2]:
population.head()

,Zone,Annee,Valeur
0,Afghanistan,2013,32269.589
1,Afghanistan,2014,33370.794
2,Afghanistan,2015,34413.603
3,Afghanistan,2016,35383.032
4,Afghanistan,2017,36296.113


In [3]:
len(population.Zone.unique())

236

In [4]:
population.Annee.unique()

array([2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

La table *population* contient le nombre d'habitants (en 1000) de 236 pays, de 2013 à 2018. 

J'ai besoin des informations pour l’année 2017, je vais donc créer la table *population_17*.  

In [5]:
population_17 = population[population['Annee']==2017]
population_17 = population_17.drop(columns=['Annee'])
population_17 = population_17.rename(columns={'Valeur': 'Population'})

In [6]:
population_17['Population'].sum()

7548134.1110000005

### Sous-nutrition

In [7]:
sous_nutrition.head()

,Zone,Annee,Valeur
0,Afghanistan,2012-2014,8.6
1,Afghanistan,2013-2015,8.8
2,Afghanistan,2014-2016,8.9
3,Afghanistan,2015-2017,9.7
4,Afghanistan,2016-2018,10.5


La table *sous_nutrition* montre le nombre de personnes en sous-alimentation en millions d’habitants, en intervalle de trois années. Pour definir la sous-nutrition en 2017, je vais prendre la moyenne de 2016-2018. 

In [8]:
sous_nutrition_17 = sous_nutrition[sous_nutrition['Annee']=="2016-2018"]
sous_nutrition_17 = sous_nutrition_17.replace({'Valeur': {'<0.1':'0'}})
sous_nutrition_17['Valeur'] = sous_nutrition_17['Valeur'].astype(float)
sous_nutrition_17['Valeur'] = sous_nutrition_17['Valeur'].fillna(0)
sous_nutrition_17 = sous_nutrition_17.replace({'Annee': {"2016-2018": "2017"}})
sous_nutrition_17.sort_values(by='Valeur', ascending=False).head()

,Zone,Annee,Valeur
508,Inde,2017,190.1
820,Pakistan,2017,24.8
514,Indonesie,2017,23.6
772,Nigeria,2017,22.8
100,Bangladesh,2017,21.5


### Aide alimentaire

In [9]:
aide_alimentaire.head()

,Pays beneficiaire,Annee,Produit,Valeur
0,Afghanistan,2013,Autres non-cereales,682
1,Afghanistan,2014,Autres non-cereales,335
2,Afghanistan,2013,Ble et Farin,39224
3,Afghanistan,2014,Ble et Farin,15160
4,Afghanistan,2013,Cereales,40504


In [10]:
aide_alimentaire = aide_alimentaire.rename(columns={'Pays beneficiaire':'Zone'})

In [11]:
aide_alimentaire.Annee.unique()

array([2013, 2014, 2015, 2016], dtype=int64)

In [12]:
len(aide_alimentaire.Zone.unique())

76

In [13]:
aide_alimentaire.Produit.unique()

array(['Autres non-cereales', 'Ble et Farin', 'Cereales',
       'Fruits secs, total', 'Huiles vegetales',
       'Legumineuses Seches,Tot.', 'Non-cereales', 'Riz, total',
       'Sucre, total', 'Cereales Secondaires', 'Lait, total',
       'Melanges et preparations', 'Poiss&produi', 'BulgurFarEnt',
       'Viande Total', 'Graisses Com'], dtype=object)

La table *aide_alimentaire* contient l'information sur la quantité qui a été donnée comme aide alimentaire (en tonnes) à 76 pays entre 2013 et 2016. 

### Disponibilité alimentaire

In [14]:
for col in dispo_alimentaire.columns:
    print(col)

Zone
Produit
Origine
Aliments pour animaux
Autres Utilisations
Disponibilite alimentaire (Kcal/personne/jour)
Disponibilite alimentaire en quantite (kg/personne/an)
Disponibilite de matiere grasse en quantite (g/personne/jour)
Disponibilite de proteines en quantite (g/personne/jour)
Disponibilite interieure
Exportations - Quantite
Importations - Quantite
Nourriture
Pertes
Production
Semences
Traitement
Variation de stock


In [15]:
len(dispo_alimentaire.Zone.unique())

174

In [16]:
len(dispo_alimentaire.Produit.unique())

98

La table *dispo_alimentaire* montre la disponibilité alimentaire de 98 produits pour l’année 2017 pour 174 pays. 

La table contient 17 variables :
-	Le pays.

-	Le produit (98 type).

-   Une variable type qui précise si le produit est d’origine animale ou végétale.

-	4 variables de disponibilité alimentaire (unités précisées directement dans le nom de la variable).

-	10 variables qui précisent les utilisations/moyens d’acquisition de la disponibilité alimentaire (en milliers de tonnes).



In [17]:
dispo_alimentaire = dispo_alimentaire.fillna(0)

In [18]:
dispo_alimentaire = dispo_alimentaire.replace({'Zone':{'Tchequie (la)':'Tchequie'}})
dispo_alimentaire = dispo_alimentaire.replace({'Zone':{'Royaume-Uni':"Royaume-Uni de Grande-Bretagne et d'Irlande du Nord"}})

## 1. La proportion de personnes en état de sous-nutrition

Pour calculer la proportion je vais ajouter la population à la table de sous-nutrition.

In [19]:
sous_nutrition_17 = pd.merge(sous_nutrition_17, population_17, how='left', on='Zone')
sous_nutrition_17['Pourcent sous-nutrition'] = 100*1000*sous_nutrition_17['Valeur']/sous_nutrition_17['Population']

Les 5 pays avec le pourcentage de la population en sous-nutrition le plus elevé:

In [20]:
sous_nutrition_17.sort_values(by='Pourcent sous-nutrition', ascending=False).head()

,Zone,Annee,Valeur,Population,Pourcent sous-nutrition
78,Haiti,2017,5.3,10982.366,48.259182
157,Republique populaire democratique de Coree,2017,12.0,25429.825,47.188685
108,Madagascar,2017,10.5,25570.512,41.062924
103,Liberia,2017,1.8,4702.226,38.279742
100,Lesotho,2017,0.8,2091.534,38.249438


In [21]:
sous_nutrition_17['Valeur'].sum()

535.7

In [22]:
100*1000*sous_nutrition_17['Valeur'].sum()/sous_nutrition_17['Population'].sum()

7.1011968332354165

Le nombre total de population en sous-nutrition en 2017 est 535.7 million, ou 7.10% de la planete.

## 2. Le nombre théorique de personnes qui pourraient être nourries

Je vais creer la table *dispo_personne* qui montre la disponibilité alimentaire total par personne, pour chaque pays.

In [23]:
dispo_personne = pd.pivot_table(dispo_alimentaire, values=['Disponibilite alimentaire (Kcal/personne/jour)','Disponibilite alimentaire en quantite (kg/personne/an)'], index=['Zone'],  aggfunc=sum)

In [24]:
dispo_personne = pd.merge(dispo_personne, population_17, how='left', on='Zone')

In [25]:
dispo_personne.sort_values(by=['Disponibilite alimentaire (Kcal/personne/jour)'])

,Zone,Disponibilite alimentaire (Kcal/personne/jour),Disponibilite alimentaire en quantite (kg/personne/an),Population
131,Republique centrafricaine,1879.0,453.73,4596.023
172,Zambie,1924.0,350.12,16853.599
97,Madagascar,2056.0,425.07,25570.512
0,Afghanistan,2087.0,351.41,36296.113
70,Haiti,2089.0,397.78,10982.366
...,...,...,...,...
80,Israel,3610.0,922.14,8243.848
54,Etats-Unis d'Amerique,3682.0,1002.39,325084.756
165,Turquie,3708.0,968.15,81116.450
17,Belgique,3737.0,1002.09,11419.748


In [26]:
dispo_personne['total_kcal'] = dispo_personne['Disponibilite alimentaire (Kcal/personne/jour)']*dispo_personne['Population']*1000

In [27]:
sum(dispo_personne['Population'])/1000

7369.269325000005

In [28]:
sum(dispo_personne['total_kcal'])/2250/1000000

9414.294554189333

In [29]:
100*(sum(dispo_personne['total_kcal'])/2250/1000000)/(sum(dispo_personne['Population'])/1000)

127.75071908759864

Théoriquement, 9.4 milliard de personnes pourraient être nourries en 2017, ce qui représente 27,75% de plus de la population mondiale, en supposant qu'une personne a besoin de 2250 calories par jour en moyenne (femme 2000kcal, homme 2500kcal).

## 3. Le nombre théorique de personnes qui pourraient être nourries avec des produits végétaux

Je vais creer la table *dispo_personne_tot_vege* qui montre la disponibilité alimentaire des produits vegetaux par personne, pour chaque pays. 

In [30]:
dispo_alimentaire_vege = dispo_alimentaire[dispo_alimentaire['Origine']=="vegetale"]
dispo_personne_vege = pd.pivot_table(dispo_alimentaire_vege, values=['Disponibilite alimentaire (Kcal/personne/jour)'], index=['Zone'],  aggfunc=sum)
dispo_personne_tot_vege = pd.merge(dispo_personne, dispo_personne_vege, on='Zone')
dispo_personne_tot_vege = dispo_personne_tot_vege.drop(columns=['Disponibilite alimentaire en quantite (kg/personne/an)'])
dispo_personne_tot_vege = dispo_personne_tot_vege.rename(columns={'Disponibilite alimentaire (Kcal/personne/jour)_x':'Disponibilite alimentaire (Kcal/personne/jour)_total', 'Disponibilite alimentaire (Kcal/personne/jour)_y':'Disponibilite alimentaire (Kcal/personne/jour)_vege'})
dispo_personne_tot_vege['Pourcentage kcal vege'] = 100*dispo_personne_tot_vege['Disponibilite alimentaire (Kcal/personne/jour)_vege']/dispo_personne_tot_vege['Disponibilite alimentaire (Kcal/personne/jour)_total']
dispo_personne_tot_vege['total kcal vege'] = dispo_personne_tot_vege['Disponibilite alimentaire (Kcal/personne/jour)_vege']*1000*dispo_personne_tot_vege['Population']

In [31]:
sum(dispo_personne_tot_vege['total kcal vege'])/2250/1000000

7755.337519527111

In [32]:
100*(sum(dispo_personne_tot_vege['total kcal vege'])/2250/1000000)/(sum(dispo_personne['Population'])/1000)

105.23889381023683

Théoriquement, toute la population mondiale pourraient être nourries avec des produits végétaux (plus précisément, 5.24% de plus). 

In [33]:
100*sum(dispo_personne_tot_vege['Disponibilite alimentaire (Kcal/personne/jour)_vege']>=2250)/len(dispo_personne_tot_vege)

52.87356321839081

Cependant, à cause de la répartition inégale, seulement 52.87% des pays pourraient en réalité être nourris uniquement avec des produits végétaux.

## 4. L’utilisation de la disponibilité intérieure (la part qui est attribuée à l’alimentation animale, celle qui est perdue et celle qui est concrètement utilisée pour l'alimentation humaine)

Je vais creer une table *dispo_interieur* comme le tableau croisé dynamique à partir de la table *dispo_alimentaire. 

In [34]:
dispo_interieur = pd.pivot_table(dispo_alimentaire, values=['Disponibilite interieure','Production','Importations - Quantite','Exportations - Quantite','Variation de stock', 'Nourriture', 'Aliments pour animaux','Semences','Traitement','Autres Utilisations', 'Pertes' ], index=['Zone'],  aggfunc=sum )

In [35]:
dispo_interieur.head()

,Aliments pour animaux,Autres Utilisations,Disponibilite interieure,Exportations - Quantite,Importations - Quantite,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
Zone,,,,,,,,,,,
Afghanistan,768.0,415.0,13515.0,278.0,2988.0,10735.0,1135.0,11171.0,395.0,63.0,-368.0
Afrique du Sud,5309.0,876.0,61256.0,11674.0,7517.0,29812.0,2193.0,63263.0,253.0,22819.0,2153.0
Albanie,660.0,174.0,4758.0,94.0,933.0,3476.0,276.0,3964.0,55.0,113.0,-45.0
Algerie,4352.0,1707.0,42630.0,655.0,17403.0,31729.0,3753.0,26359.0,449.0,637.0,-479.0
Allemagne,30209.0,7711.0,162275.0,57431.0,65353.0,79238.0,3781.0,154547.0,1551.0,39842.0,-192.0


In [36]:
dispo_interieur['dispo_interieur calculee'] = dispo_interieur['Production']+dispo_interieur['Variation de stock']+dispo_interieur['Importations - Quantite']-dispo_interieur['Exportations - Quantite']

In [37]:
from scipy.stats import pearsonr
pearsonr(dispo_interieur['Disponibilite interieure'], dispo_interieur['dispo_interieur calculee'])

(0.9999999998062397, 0.0)

In [38]:
dispo_interieur[{'Disponibilite interieure', 'dispo_interieur calculee'}].describe()

,Disponibilite interieure,dispo_interieur calculee
count,1.740000e+02,1.740000e+02
mean,5.660341e+04,5.660444e+04
std,2.008030e+05,2.008034e+05
min,5.900000e+01,5.900000e+01
25%,2.854500e+03,2.857750e+03
50%,1.113200e+04,1.113350e+04
75%,3.122925e+04,3.122550e+04
max,2.036726e+06,2.036728e+06


Les résultats ci-dessus confirment que la formule de disponibilité alimentaire est :

$$Disponibilité\,\, interieur = Production + Importations - Exportations + Variation\,\, des\,\, Stocks$$

In [39]:
dispo_interieur['alimentation humaine'] = dispo_interieur['Disponibilite interieure']-dispo_interieur['Aliments pour animaux']-dispo_interieur['Autres Utilisations']-dispo_interieur['Semences']-dispo_interieur['Traitement']-dispo_interieur['Pertes']

In [40]:
pearsonr(dispo_interieur['Nourriture'], dispo_interieur['alimentation humaine'])

(0.9999947122144711, 0.0)

Les résultats ci-dessus confirment que la formule de disponibilité de l’alimentation humaine est : 

$$Nouriture = Disponibilite\,\, interieur - Alimentation\,\, animale - Autres\,\, usages- Semences – Traitement  – Pertes$$

En combinant les deux formules ensemble, on a l'identité suivante:

$Disponibilite\,\, interieur =$

$=Production + Importations - Exportations + Variation\,\, des\,\, Stocks$

$=Nouriture+Alimentation\,\, animale + Autres\,\, usages + Semences + Traitement + Pertes$


In [41]:
dispo_interieur['pourcent animaux'] = 100*(dispo_interieur['Aliments pour animaux'])/(dispo_interieur['Disponibilite interieure'])
dispo_interieur['pourcent pertes'] = 100*(dispo_interieur['Pertes'])/(dispo_interieur['Disponibilite interieure'])
dispo_interieur['pourcent nourriture'] = 100*(dispo_interieur['Nourriture'])/(dispo_interieur['Disponibilite interieure'])

In [42]:
dispo_interieur[{'pourcent animaux', 'pourcent pertes', 'pourcent nourriture'}].describe()

,pourcent pertes,pourcent animaux,pourcent nourriture
count,174.000000,174.000000,174.000000
mean,4.656214,12.024054,57.522398
std,4.094526,9.754659,18.520794
min,0.000000,0.000000,9.788155
25%,2.121844,3.548217,46.268607
50%,3.633931,10.223261,57.743815
75%,5.891000,17.821802,71.837118
max,33.348554,39.672895,96.083551


En moyenne, 57.5% de disponibilité alimentaire total est utilisé pour l'alimentation humaine, 12% pour l'alimentation animal, et 4.66% est perdu.

Les dix pays ou le pourcentage des pertes est le plus eleve sont:

In [43]:
pd.merge(dispo_interieur.drop(columns=['alimentation humaine','dispo_interieur calculee','Variation de stock','Traitement','Semences','Production','Importations - Quantite','Exportations - Quantite','Aliments pour animaux','Autres Utilisations',]),sous_nutrition_17[{'Zone', 'Pourcent sous-nutrition'}], how='left', on='Zone').sort_values(by='pourcent pertes',ascending=False)[0:9]

,Zone,Disponibilite interieure,Nourriture,Pertes,pourcent animaux,pourcent pertes,pourcent nourriture,Pourcent sous-nutrition
148,Sierra Leone,6570.0,3070.0,2191.0,13.713851,33.348554,46.727549,26.707893
63,Ghana,38998.0,22482.0,7442.0,12.508334,19.083030,57.649110,6.867786
30,Cameroun,26222.0,15259.0,3981.0,8.260239,15.181908,58.191595,6.513047
5,Angola,32029.0,15059.0,4799.0,28.908177,14.983296,47.016766,19.452143
46,Dominique,141.0,68.0,20.0,14.184397,14.184397,48.226950,0.000000
67,Guinee,8404.0,5711.0,1074.0,7.710614,12.779629,67.955735,0.000000
99,Malawi,19877.0,8607.0,2428.0,20.169040,12.215123,43.301303,18.675514
161,Togo,4095.0,3186.0,476.0,3.687424,11.623932,77.802198,20.783340
26,Bulgarie,9458.0,5111.0,1090.0,11.154578,11.524635,54.038909,2.815932


C'est interesant de voir que les pays ou le pourcentage des pertes est le plus eleve, sont les pays avec beaucoup de habitants en sous-nutrition.
Par contre, les pays ou il n'y a pas de famine, il n'y a meme pas beaucoup de pertes, ce que nous pouvons voir ci-desous.

In [44]:
pd.merge(dispo_interieur.drop(columns=['alimentation humaine','dispo_interieur calculee','Variation de stock','Traitement','Semences','Production','Importations - Quantite','Exportations - Quantite','Aliments pour animaux','Autres Utilisations',]),sous_nutrition_17[{'Zone', 'Pourcent sous-nutrition'}], how='left', on='Zone').sort_values(by='pourcent pertes')[0:9]

,Zone,Disponibilite interieure,Nourriture,Pertes,pourcent animaux,pourcent pertes,pourcent nourriture,Pourcent sous-nutrition
79,Islande,474.0,301.0,0.0,22.573840,0.000000,63.502110,0.0
100,Maldives,337.0,277.0,0.0,0.000000,0.000000,82.195846,0.0
141,Saint-Kitts-et-Nevis,65.0,26.0,0.0,0.000000,0.000000,40.000000,0.0
6,Antigua-et-Barbuda,79.0,58.0,0.0,0.000000,0.000000,73.417722,0.0
20,Bermudes,59.0,51.0,0.0,1.694915,0.000000,86.440678,0.0
88,Kiribati,241.0,67.0,0.0,0.000000,0.000000,27.800830,0.0
34,Chine - RAS de Macao,383.0,368.0,1.0,0.000000,0.261097,96.083551,0.0
104,Maurice,4767.0,794.0,15.0,2.139711,0.314663,16.656178,0.0
58,Finlande,10034.0,5938.0,40.0,21.467012,0.398645,59.178792,0.0


## 5. Résultats supplémentaires

In [45]:
dispo_alimentaire[(dispo_alimentaire['Zone']=='Thailande') & (dispo_alimentaire['Produit']=='Manioc')]

,Zone,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilite alimentaire (Kcal/personne/jour),Disponibilite alimentaire en quantite (kg/personne/an),Disponibilite de matiere grasse en quantite (g/personne/jour),Disponibilite de proteines en quantite (g/personne/jour),Disponibilite interieure,Exportations - Quantite,Importations - Quantite,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
13809,Thailande,Manioc,vegetale,1800.0,2081.0,40.0,13.0,0.05,0.14,6264.0,25214.0,1250.0,871.0,1511.0,30228.0,0.0,0.0,0.0


In [46]:
100*25214.0/30228.0

83.41272991928014

In [47]:
100*1250.0/6264.0

19.955300127713922

In [48]:
100*871/6264

13.90485312899106

En 2017, la Thaïlande a produit 30 228 mille tonnes de manioc, dont 83% ont été exportées. Par contre, ils ont dû importer 1250 mille tonnes (ou 20% de disponibilité interieur total) pour pouvoir couvrir les besoins.

Seulement 14% de la disponibilité intérieure est utilisée pour l'alimentation humaine.

In [49]:
dispo_alimentaire[(dispo_alimentaire['Production']>30000) & (dispo_alimentaire['Produit']=='Manioc')][{'Zone', 'Production', 'Disponibilite interieure', 'Exportations - Quantite', 'Importations - Quantite'}]

,Zone,Exportations - Quantite,Importations - Quantite,Production,Disponibilite interieure
9830,Nigeria,11.0,15.0,53000.0,53004.0
13809,Thailande,25214.0,1250.0,30228.0,6264.0


Seul le Nigéria produit plus de manioc que la Thaïlande, mais tout est utilisé en interne.

## 6. Les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017

In [50]:
sous_nutrition_17[{'Zone', 'Pourcent sous-nutrition'}].sort_values(by='Pourcent sous-nutrition', ascending=False)[0:19]

,Zone,Pourcent sous-nutrition
78,Haiti,48.259182
157,Republique populaire democratique de Coree,47.188685
108,Madagascar,41.062924
103,Liberia,38.279742
100,Lesotho,38.249438
183,Tchad,37.957606
161,Rwanda,35.055619
121,Mozambique,32.810898
186,Timor-Leste,32.173531
0,Afghanistan,28.928718


Presque 50% de population de Haiti et Republique populaire democratique de Coree est en sous-nutrition, suivi par Madagascar (41%), Liberia et Lesotho (38%), etc. 

Entre les premiers 20 pays avec une sous-nutrition tres eleve, plus que une moitie sont des pays africaines. 


In [51]:
sous_nutrition_17['Pourcent sous-nutrition'].mean()

5.845468875057933

Le pourcentage moyen de la population en sous-nutrition par pays est 5.85%. 

In [52]:
sum(sous_nutrition_17['Pourcent sous-nutrition']>10)

42

Le pourcentage de la population en sous-nutrition est plus que 10% dans 42 pays au monde.

## 7. Les pays qui ont le plus bénéficié d’aide depuis 2013 

Je vais creer la table *aide_total* pour calculer l'aide total reçu depuis 2013.

In [53]:
aide_total=pd.pivot_table(aide_alimentaire, values='Valeur', index=['Zone'],  aggfunc=sum).sort_values(by=['Valeur'], ascending=False)

15 pays qui ont reçu le plus d'aide:

In [54]:
aide_total[0:14]

,Valeur
Zone,
Republique arabe syrienne,1858943
Ethiopie,1381294
Yemen,1206484
Soudan du Sud,695248
Soudan,669784
Kenya,552836
Bangladesh,348188
Somalie,292678
Republique democratique du Congo,288502


In [55]:
aide_personne = pd.merge(sous_nutrition_17[{'Zone', 'Population'}],aide_total, how='right', on='Zone')

In [56]:
aide_personne['aide/personne'] = aide_personne['Valeur']/aide_personne['Population']

15 pays qui ont reçu le plus d'aide par personne:

In [57]:
aide_personne.sort_values(by='aide/personne', ascending=False)[0:14]

,Zone,Population,Valeur,aide/personne
0,Republique arabe syrienne,17095.678,1858943,108.737600
3,Soudan du Sud,10910.763,695248,63.721300
37,Djibouti,944.099,41296,43.741175
2,Yemen,27834.819,1206484,43.344417
16,Palestine,4747.227,169684,35.743814
7,Somalie,14589.179,292678,20.061307
19,Nicaragua,6384.846,115332,18.063396
10,Tchad,15016.753,267966,17.844470
4,Soudan,40813.397,669784,16.410886
27,Republique centrafricaine,4596.023,66610,14.492965


## 8. Les pays ayant le plus/le moins de disponibilité/habitant, etc.

In [58]:
disponibilite = pd.pivot_table(dispo_alimentaire, index=['Zone'], aggfunc=sum)

In [59]:
disponibilite = pd.merge(disponibilite, sous_nutrition_17, how='left', on='Zone')

10 pays avec le plus de  disponibilité/habitant:

In [60]:
disponibilite[{'Zone', 'Disponibilite alimentaire (Kcal/personne/jour)', 'Pourcent sous-nutrition'}].sort_values(by='Disponibilite alimentaire (Kcal/personne/jour)', ascending=False)[0:9]

,Zone,Pourcent sous-nutrition,Disponibilite alimentaire (Kcal/personne/jour)
11,Autriche,0.000000,3770.0
17,Belgique,0.000000,3737.0
165,Turquie,0.000000,3708.0
54,Etats-Unis d'Amerique,0.000000,3682.0
80,Israel,0.000000,3610.0
78,Irlande,0.000000,3602.0
81,Italie,0.000000,3578.0
95,Luxembourg,0.000000,3540.0
47,Egypte,4.769677,3518.0


10 pays avec le moins de disponibilité/habitant:

In [61]:
disponibilite[{'Zone', 'Disponibilite alimentaire (Kcal/personne/jour)', 'Pourcent sous-nutrition'}].sort_values(by='Disponibilite alimentaire (Kcal/personne/jour)')[0:20]

,Zone,Pourcent sous-nutrition,Disponibilite alimentaire (Kcal/personne/jour)
131,Republique centrafricaine,0.000000,1879.0
172,Zambie,0.000000,1924.0
97,Madagascar,41.062924,2056.0
0,Afghanistan,28.928718,2087.0
70,Haiti,48.259182,2089.0
136,Republique populaire democratique de Coree,47.188685,2093.0
157,Tchad,37.957606,2109.0
173,Zimbabwe,0.000000,2113.0
120,Ouganda,0.000000,2126.0
55,Ethiopie,19.830841,2129.0


In [62]:
sum(disponibilite['Disponibilite alimentaire (Kcal/personne/jour)']<2250)

19

In [63]:
sum(disponibilite['Pourcent sous-nutrition']>0)

84

Il n'y a que 19 pays où la disponibilité alimentaire (en kcal par personne et par jour) est inférieure à 2250, mais en réalité il y a au moins 84 pays où le pourcentage de personnes en sous-nutrition est supérieur à zéro (ou plus que 84, parce que il y a des valuers manquants pour la sous-nutrition).

# CONCLUSION

En 2017, la population mondiale était de 7,4 milliards.

Même s'il y avait assez de nourriture dans le monde pour nourrir 9,4 milliards de personnes, plus de 7% de la population mondiale était en sous-nutrition. 

La famine était présente dans plus de 80 pays, avec Haïti, la Corée du Nord, Madagascar, Liberia, Lesotho, Tchad, Rwanda, Mozambique, Timor-Leste et Afghanistan étant les plus en difficulté.

76 pays ont reçu une aide alimentaire de 2013 à 2017, parmi lesquels Syria, Ethiopie, Yemen, Soudan du Sud, Soudan, Kenya, Bangladesh, Somalie, Congo et Niger ont le plus bénéficié d’aide.

En conclusion, le problème du monde n'est pas le manque de nourriture, mais la distribution inégale: certaines parties du monde ont trop de nourriture alors que d'autres n'en ont pas assez.